# Constant-time extended GCD

## Outline :

1. SageMath System and GCD

2. DivStep and constant-time GCD

3. JumpDivStep and constant-time GCD


## Section 1: SageMath System and GCD

### SageMath: http://www.sagemath.org/



In [69]:
#The enviroments:

p = 761; q61 = 765; q = 6*q61+1; w = 250
Zx.<x> = ZZ[]; R.<xp> = Zx.quotient(x^p-x-1)
Fq = GF(q); Fqx.<xq> = Fq[]; Rq.<xqp> = Fqx.quotient(x^p-x-1)
F3 = GF(3); F3x.<x3> = F3[]; R3.<x3p> = F3x.quotient(x^p-x-1)

In [70]:

# functions for lists

def conditional_swap( condition , a , b ) :
  if 0 == condition:
    return a , b
  else :
    return b , a


def list_shift(l,shift) :
  src_idx = max(-shift,0)
  dst_idx = max(shift,0)
  length = max(len(l)-abs(shift),0)
  new_l = [ l[0].parent()(0) ]* len(l)
  new_l[dst_idx:dst_idx+length] = l[src_idx:src_idx+length]
  return new_l


def list_pad_zero( al , len ):
  al_ref = [ al[0].parent(0) ]*(len)
  al = map( lambda x,y: y if x is None else x, al , al_ref )
  return al


def list_pad_front(al,nlen):
  assert nlen >= len(al)
  ral = list(al)
  ral[:0] = [al[0].parent(0)]*(nlen-len(al))
  return ral


def list_mul( l , c ):
  return [ x*c for x in l ]

def list_add( l1 , l2 ):
  return [ x+y for x,y in zip(l1,l2) ]


###################################

# functions for polynomials(lists)

def poly_in_deg( a , deg ):
  return list_pad_zero( a.list() , deg+1 )


def poly_rev( a , deg ):
  r = poly_in_deg(a,deg)
  rev_r = r[::-1]
  return rev_r


def poly_mul( a , b ):
  r = [a[0].parent(0)]*(len(a)+len(b)-1)
  for i in range(len(a)):
    r[i:i+len(b)] = list_add( r[i:i+len(b)] , list_mul(b,a[i]) )
  return r


###################################
##
##  This is not for a normal matrix. 
##  It is for the "transition matrix".
##
###################################

def mat_mul_vec( mat , vec ) :
  m_l = len(mat[0][0])
  f_m00 = poly_mul(mat[0][0],vec[0])
  g_m01 = poly_mul(mat[0][1],vec[1])
  f = list_add( f_m00 , g_m01 )
  f_m10 = poly_mul(mat[1][0],vec[0])
  g_m11 = poly_mul(mat[1][1],vec[1])
  g = list_add( f_m10 , g_m11 )
  ## XXX: assert 0 for truncated part.
  rf = f[m_l-1:]
  rg = list_pad_zero( g[m_l:] , len(vec[1]) )
  return  rf , rg 


def mat_mul( mat1 , mat2 ) :
  m1_len = len(mat1[0][0])
  m2_len = len(mat2[0][0])
  r_len = m1_len+m2_len
  r00_u = poly_mul(mat1[0][0],mat2[0][0])
  r00_d = poly_mul(mat1[0][1],mat2[1][0])
  r00 = list_add(list_pad_front(r00_u,r_len),list_pad_zero(r00_d,r_len))
  r01_u = poly_mul(mat1[0][0],mat2[0][1])
  r01_d = poly_mul(mat1[0][1],mat2[1][1])
  r01 = list_add(list_pad_front(r01_u,r_len),list_pad_zero(r01_d,r_len))
  r10_u = poly_mul(mat1[1][0],mat2[0][0])
  r10_d = poly_mul(mat1[1][1],mat2[1][0])
  r10 = list_add(list_pad_front(r10_u,r_len),list_pad_zero(r10_d,r_len))
  r11_u = poly_mul(mat1[1][0],mat2[0][1])
  r11_d = poly_mul(mat1[1][1],mat2[1][1])
  r11 = list_add(list_pad_front(r11_u,r_len),list_pad_zero(r11_d,r_len))
  return [ [r00,r01] , [ r10 , r11 ] ]

def deg1mat_mul( deg1mat , mat2 ) :
  inp_poly_len = len(mat2[0][0])
  np_len = inp_poly_len+1
  mat2_r0 = [list_pad_front(mat2[0][0],np_len),list_pad_front(mat2[0][1],np_len)]
  mat2_r1 = [list_pad_zero(mat2[1][0],np_len),list_pad_zero(mat2[1][1],np_len)]
  r0 , r0_ = conditional_swap( deg1mat[0][0][0] , mat2_r1 , mat2_r0 )   # use AND
  m1_10 = deg1mat[1][0][0]
  m1_11 = deg1mat[1][1][0]
  r10_u = list_mul(mat2[0][0],m1_10)
  r10_d = list_mul(mat2[1][0],m1_11)
  r11_u = list_mul(mat2[0][1],m1_10)
  r11_d = list_mul(mat2[1][1],m1_11)
  r10 = list_add(list_pad_front(r10_u,np_len),list_pad_zero(r10_d,np_len))
  r11 = list_add(list_pad_front(r11_u,np_len),list_pad_zero(r11_d,np_len))
  return [ r0 , [ r10 , r11 ] ]


In [71]:
# Some variables for testing

a1 = Fqx( xq*Fq.random_element() + Fq.random_element() )
b1 = Fqx( xq*Fq.random_element() + Fq.random_element() )
c1 = Fqx( xq*Fq.random_element() + Fq.random_element() )
a2 = Fqx.random_element()
b2 = Fqx.random_element()
c2 = Fqx.random_element()
a3 = Fqx( xq*Fq.random_element() + Fq.random_element() )*Fqx.random_element()
b3 = Fqx( xq*Fq.random_element() + Fq.random_element() )*Fqx.random_element()
c3 = Fqx( xq*Fq.random_element() + Fq.random_element() )*Fqx.random_element()

In [72]:
print a1,b1,c1
print a2,b2,c2
print a3,b3,c3

1500*xq + 706 602*xq + 2491 1748*xq + 2442
483*xq^2 + 3300*xq + 276 3443*xq^2 + 139*xq + 1223 1284*xq^2 + 4059*xq + 4589
3870*xq^3 + 1686*xq^2 + 836*xq + 3419 3063*xq^3 + 4024*xq^2 + 1589*xq + 4055 1499*xq^3 + 724*xq^2 + 1680*xq + 4275


In [73]:
# The standard GCD in the textbook

#output: gcd = s*f + t*g
def ext_gcd( f , g ) :
  if f.degree() < g.degree() :  f,g = g,f  #swap
  T = f.parent()
  lr = [ f , g ]
  ls = [ T(1) , T(0) ]
  lt = [ T(0) , T(1) ]
  print "lr: [f,g] = ", lr,'\nls:',ls, "\nlt:", lt, '\n'
  while lr[-1] != 0 :
    qq = lr[-2] // lr[-1]
    rr = lr[-2] % lr[-1]
    lr.append( rr )
    ls.append( ls[-2] - qq*ls[-1] )
    lt.append( lt[-2] - qq*lt[-1] )
    print "lr[-1]:",lr[-1] , "\tls[-1]:", ls[-1], "\tlt[-1]" , lt[-1]
  print "\nlr:", lr,'\nls:',ls, "\nlt:", lt, '\n'
  #normalize
  inv_n = (lr[-2].coefficients()[0])^-1
  return lr[-2]*inv_n,ls[-2]*inv_n,lt[-2]*inv_n

ext_gcd( a2, b3 )

lr: [f,g] =  [3063*xq^3 + 4024*xq^2 + 1589*xq + 4055, 483*xq^2 + 3300*xq + 276] 
ls: [1, 0] 
lt: [0, 1] 

lr[-1]: 2895*xq + 1432 	ls[-1]: 1 	lt[-1] 621*xq + 1604
lr[-1]: 1118 	ls[-1]: 3430*xq + 7 	lt[-1] 4397*xq^2 + 1458*xq + 2047
lr[-1]: 0 	ls[-1]: 1064*xq^2 + 3477*xq + 608 	lt[-1] 4231*xq^3 + 251*xq^2 + 2754*xq + 715

lr: [3063*xq^3 + 4024*xq^2 + 1589*xq + 4055, 483*xq^2 + 3300*xq + 276, 2895*xq + 1432, 1118, 0] 
ls: [1, 0, 1, 3430*xq + 7, 1064*xq^2 + 3477*xq + 608] 
lt: [0, 1, 621*xq + 1604, 4397*xq^2 + 1458*xq + 2047, 4231*xq^3 + 251*xq^2 + 2754*xq + 715] 



(1, 1235*xq + 1080, 238*xq^2 + 2613*xq + 2979)

In [74]:
#first variant: Performing operations as matrix-multiplication of 2x2 matrices.

def ext_gcd1( f , g ) :
  if f.degree() < g.degree() :  f,g = g,f  #swap
  Ta = f.parent()
  x = Ta.gen()
  lr = [ f , g ]
  ls = [Ta(1),Ta(0)]
  lt = [Ta(0),Ta(1)]
  print "[f,g]:" , lr,"\n",ls,"\n",lt , "\n"
  while lr[-1] != 0 :
    if lr[0].degree() > lr[1].degree() : 
       lr = lr[::-1]
       ls , lt = lt , ls
       print "op0: [0,1] <-- swap, choose g"
    else:
       print "op0: [1,0] <-- choose f"
    delta = lr[1].degree() - lr[0].degree()
    lc_f = lr[0].leading_coefficient()
    lc_g = lr[1].leading_coefficient()
    lr[1] = lr[1]*lc_f - lr[0]*lc_g*(x^delta)
    lt[0] = lt[0]*lc_f-ls[0]*lc_g*(x^delta)
    lt[1] = lt[1]*lc_f-ls[1]*lc_g*(x^delta)
    print "op1: [" , lc_f , "," , lc_g , "x^" , delta , "]"
    print "-->\n[f,g]:" , lr,"\n",ls,"\n",lt , "\n"
  #normalize
  inv_n = (lr[-2].leading_coefficient())^-1
  return lr[-2]*inv_n,map(lambda x:x*inv_n, ls )

# demo code:
ext_gcd1( a2, b2 )

[f,g]: [483*xq^2 + 3300*xq + 276, 3443*xq^2 + 139*xq + 1223] 
[1, 0] 
[0, 1] 

op0: [1,0] <-- choose f
op1: [ 483 , 3443 x^ 0 ]
-->
[f,g]: [483*xq^2 + 3300*xq + 276, 3688*xq + 3130] 
[1, 0] 
[1148, 483] 

op0: [0,1] <-- swap, choose g
op1: [ 3688 , 483 x^ 1 ]
-->
[f,g]: [3688*xq + 3130, 2899*xq + 3277] 
[1148, 483] 
[1027*xq + 3688, 852*xq] 

op0: [1,0] <-- choose f
op1: [ 3688 , 2899 x^ 0 ]
-->
[f,g]: [3688*xq + 3130, 10] 
[1148, 483] 
[xq + 3225, 1932*xq + 38] 

op0: [0,1] <-- swap, choose g
op1: [ 10 , 3688 x^ 1 ]
-->
[f,g]: [10, 3754] 
[xq + 3225, 1932*xq + 38] 
[903*xq^2 + 1481*xq + 2298, 16*xq^2 + 2177*xq + 239] 

op0: [1,0] <-- choose f
op1: [ 10 , 3754 x^ 0 ]
-->
[f,g]: [10, 0] 
[xq + 3225, 1932*xq + 38] 
[4439*xq^2 + 1874*xq + 4433, 160*xq^2 + 4458*xq + 2059] 



(1, [4132*xq + 2618, 3866*xq + 922])

## Section 2: DivStep and constant-time GCD

In [75]:

# One 'division step' in the GCD.
# Trying to make it time-constant by applying 'conditional swap'

def divstep( delta , f , g ):
  coef_k = f[0].parent()
  kx.<x> = coef_k[] #kx = T_[0][0].parent()
  #x = kx.gen()
  #M2kx = T_.parent() 
  M2kx = MatrixSpace(kx.fraction_field(),2)   
  f = list_pad_zero( f , max(len(f),len(g)))
  g = list_pad_zero( g , max(len(f),len(g)))
  T0 = M2kx( (1,0,-g[0]/x,f[0]/x) )
  T1 = M2kx( (0,1,g[0]/x,-f[0]/x) )
  #conditional move
  c = 1 if delta > 0 and g[0] != 0 else 0
  rc , r1 = conditional_swap( c , [delta,f,g] , [-delta,g,f] )
  T0 , T1 = conditional_swap( c , T0 , T1 )
  # elimination
  rc[0] = 1 + rc[0]
  f = rc[1]
  g = rc[2]
  rg = [ f[0] * gi - g[0] * fi for fi,gi in zip(f,g) ]
  #rg.pop(0)
  rg = list_shift(rg,-1)
  return rc[0] , rc[1] , rg , T0

# demo code:
# Warning: The lists are in the reverse order of polynomials
#
f = poly_rev( a2 , 2 )
g = poly_rev( b2 , 2 )
print "[f,g] = ", [ f, g ] , "\ndivstep(0,f,g): -->"
divstep( 0 , f , g )



[f,g] =  [[483, 3300, 276], [3443, 139, 1223]] 
divstep(0,f,g): -->


(
                                      [     1      0]
1, [483, 3300, 276], [3688, 3130, 0], [1148/x  483/x]
)

In [76]:

# The reversed Constant-time GCD

def ext_gcd2( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  #if deg_a == deg_b : 
  #  b = b*a.leading_coefficient() - a*b.leading_coefficient()
  #deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  M2kx = MatrixSpace( a.parent().fraction_field() , 2 )
  T0 = M2kx(1)
  print delta , r1 , r2 , "\n", T0 , "\n"
  delta , r1 , r2 , T1 = divstep( delta , r1 , r2 )
  T0 = M2kx(T1)*T0
  print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep( delta , r1 , r2 )
    T0 = M2kx(T1)*T0
    print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0] = T0[0]*inv
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0

# demo code:
ext_gcd2( a2, b2 )


0 [483, 3300, 276] [3443, 139, 1223] 
[1 0]
[0 1] 

1 [483, 3300, 276] [3688, 3130, 0] 
[     1      0]
[1148/x  483/x] 
[      1       0]
[1148/xq  483/xq] 

0 [3688, 3130, 0] [2899, 3277, 0] 
[     0      1]
[3688/x 4108/x] 
[              1148/xq                483/xq]
[(3688*xq + 1027)/xq^2              852/xq^2] 

1 [3688, 3130, 0] [10, 0, 0] 
[     1      0]
[1692/x 3688/x] 
[            1148/xq              483/xq]
[ (3225*xq + 1)/xq^3 (38*xq + 1932)/xq^3] 

0 [10, 0, 0] [3754, 0, 0] 
[    0     1]
[ 10/x 903/x] 
[              (3225*xq + 1)/xq^3              (38*xq + 1932)/xq^3]
[(2298*xq^2 + 1481*xq + 903)/xq^4   (239*xq^2 + 2177*xq + 16)/xq^4] 

output:
0 [1, 0, 0] [3754, 0, 0] 
[           (2618*xq + 4132)/xq^3             (922*xq + 3866)/xq^3]
[(2298*xq^2 + 1481*xq + 903)/xq^4   (239*xq^2 + 2177*xq + 16)/xq^4] 



(
[1, 0, 0],

[           (2618*xq + 4132)/xq^3             (922*xq + 3866)/xq^3]
[(2298*xq^2 + 1481*xq + 903)/xq^4   (239*xq^2 + 2177*xq + 16)/xq^4]
)

In [77]:
ext_gcd2( a2*b1 , a1*b1 )

1 [1533, 3599, 3286, 3457] [3164, 2066, 293] 
[1 0]
[0 1] 

0 [3164, 2066, 293, 0] [2168, 3629, 2186, 0] 
[     0      1]
[3164/x 3058/x] 
[      0       1]
[3164/xq 3058/xq] 

1 [3164, 2066, 293, 0] [1793, 792, 0, 0] 
[     1      0]
[2423/x 3164/x] 
[                    0                     1]
[            2516/xq^2 (2423*xq + 2275)/xq^2] 

0 [1793, 792, 0, 0] [199, 1975, 0, 0] 
[     0      1]
[1793/x 1427/x] 
[                      2516/xq^2           (2423*xq + 2275)/xq^2]
[                       170/xq^3 (1793*xq^2 + 598*xq + 588)/xq^3] 

1 [1793, 792, 0, 0] [0, 0, 0, 0] 
[     1      0]
[4392/x 1793/x] 
[                        2516/xq^2             (2423*xq + 2275)/xq^2]
[            (4326*xq + 1804)/xq^4 (1027*xq^2 + 4295*xq + 2945)/xq^4] 

2 [1793, 792, 0, 0] [0, 0, 0, 0] 
[     1      0]
[     0 1793/x] 
[                      2516/xq^2           (2423*xq + 2275)/xq^2]
[          (2319*xq + 2508)/xq^5 (420*xq^2 + 1828*xq + 735)/xq^5] 

output:
2 [1, 2986, 0, 0] [0, 0, 0, 0]

(
[1, 2986, 0, 0],

[                      3179/xq^2           (3837*xq + 3650)/xq^2]
[          (2319*xq + 2508)/xq^5 (420*xq^2 + 1828*xq + 735)/xq^5]
)

In [78]:
ext_gcd1( a2*b1 , a1*b1 )

[f,g]: [1533*xq^3 + 3599*xq^2 + 3286*xq + 3457, 3164*xq^2 + 2066*xq + 293] 
[1, 0] 
[0, 1] 

op0: [0,1] <-- swap, choose g
op1: [ 3164 , 1533 x^ 1 ]
-->
[f,g]: [3164*xq^2 + 2066*xq + 293, 2168*xq^2 + 3629*xq + 2186] 
[0, 1] 
[3164, 3058*xq] 

op0: [1,0] <-- choose f
op1: [ 3164 , 2168 x^ 0 ]
-->
[f,g]: [3164*xq^2 + 2066*xq + 293, 1793*xq + 792] 
[0, 1] 
[2516, 2275*xq + 2423] 

op0: [0,1] <-- swap, choose g
op1: [ 1793 , 3164 x^ 1 ]
-->
[f,g]: [1793*xq + 792, 199*xq + 1975] 
[2516, 2275*xq + 2423] 
[170*xq, 588*xq^2 + 598*xq + 1793] 

op0: [1,0] <-- choose f
op1: [ 1793 , 199 x^ 0 ]
-->
[f,g]: [1793*xq + 792, 0] 
[2516, 2275*xq + 2423] 
[1804*xq + 4326, 2945*xq^2 + 4295*xq + 1027] 



(xq + 2986, [3179, 3650*xq + 3837])

In [79]:
# The "list" version of previous ext_ext2()

def divstep_list( delta , f , g ):
  coef_k = f[0].parent()
  T00 = [ [coef_k(1)] , [coef_k(0)] ]
  T10 = [ [-g[0]] , [f[0]]  ]
  T01 = [ [coef_k(0)] , [coef_k(1)] ]
  T11 = [ [g[0]]  , [-f[0]] ]
  #conditional move
  c = 1 if delta > 0 and g[0] != 0 else 0
  rd , rd_ = conditional_swap( c , delta , -delta )
  f  , g   = conditional_swap( c , f , g )
  T0 , T0_ = conditional_swap( c , [ T00, T10 ] , [ T01 , T11 ] )
  # elimination
  rd = rd + 1
  rg = [ f[0] * gi - g[0] * fi for fi,gi in zip(f,g) ]
  rg = list_shift( rg , -1 )
  return rd , f , rg , T0

f = poly_rev( a2 , 2 )
g = poly_rev( b2 , 2 )
print "[f,g] = ", [ f, g ] , "\ndivstep(0,f,g): -->"
divstep_list( 0 , f , g )


[f,g] =  [[483, 3300, 276], [3443, 139, 1223]] 
divstep(0,f,g): -->


(1, [483, 3300, 276], [3688, 3130, 0], [[[1], [0]], [[1148], [483]]])

In [80]:

def extgcd_list( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  if deg_a == deg_b : 
    b = b*a.leading_coefficient() - a*b.leading_coefficient()
  deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  lenlist = deg_a + 1
  r2 = list_pad_zero( r2 , lenlist )
  coef_k = r1[0].parent()
  print delta , r1 , r2 , "\n"
  delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
  T0 = T1    # T1*Identity = T1
  print delta , r1 , r2 , '\n' , T1 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
    T0 = deg1mat_mul( T1 , T0 )
    print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0][0] = list_mul( T0[0][0] , inv )
  T0[0][1] = list_mul( T0[0][1] , inv )
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0


# demo code:
extgcd_list( a2*b1 , a1*b1 )



1 [1533, 3599, 3286, 3457] [3164, 2066, 293, 0] 

0 [3164, 2066, 293, 0] [2168, 3629, 2186, 0] 
[[[0], [1]], [[3164], [3058]]] 

1 [3164, 2066, 293, 0] [1793, 792, 0, 0] 
[[[1], [0]], [[2423], [3164]]] 
[[[0, 0], [0, 1]], [[2516, 0], [2275, 2423]]] 

0 [1793, 792, 0, 0] [199, 1975, 0, 0] 
[[[0], [1]], [[1793], [1427]]] 
[[[2516, 0, 0], [2275, 2423, 0]], [[170, 0, 0], [588, 598, 1793]]] 

1 [1793, 792, 0, 0] [0, 0, 0, 0] 
[[[1], [0]], [[4392], [1793]]] 
[[[0, 2516, 0, 0], [0, 2275, 2423, 0]], [[1804, 4326, 0, 0], [2945, 4295, 1027, 0]]] 

2 [1793, 792, 0, 0] [0, 0, 0, 0] 
[[[1], [0]], [[0], [1793]]] 
[[[0, 0, 2516, 0, 0], [0, 0, 2275, 2423, 0]], [[2508, 2319, 0, 0, 0], [735, 1828, 420, 0, 0]]] 

output:
2 [1, 2986, 0, 0] [0, 0, 0, 0] 
[[[0, 0, 3179, 0, 0], [0, 0, 3650, 3837, 0]], [[2508, 2319, 0, 0, 0], [735, 1828, 420, 0, 0]]] 



([1, 2986, 0, 0],
 [[[0, 0, 3179, 0, 0], [0, 0, 3650, 3837, 0]],
  [[2508, 2319, 0, 0, 0], [735, 1828, 420, 0, 0]]])

## Section 3: JumpDivStep and constant-time GCD

In [81]:


def jumpdivsteps(n,t,delta,f,g):
  print "jumpdivsteps(",n,t,delta,f,g,")"
  assert t>= n and n>=0
  f,g = f.truncate(t), g.truncate(t)
  print "truncated: f=",f , "  g=", g
  kx=f.parent()
  x=kx.gen()
  M2kx=MatrixSpace(kx.fraction_field(),2)
  if n == 0: return delta,f,g,(),M2kx(1)
  if n==1:
    c = 1 if delta > 0 and g[0] != 0 else 0
    Tc, Td = conditional_swap( c , M2kx((1,0,-g[0]/x,f[0]/x)), M2kx( (0,1,g[0]/x,-f[0]/x) ) )
    delta , d2 = conditional_swap( c , delta , -delta )
    fc , f2 = conditional_swap( c , f , g )
    gc , g2 = conditional_swap( c , f[0]*g-g[0]*f , g[0]*f-f[0]*g )
    return 1+delta,fc,kx(gc/x),(Tc,),Tc
  # other cases of n
  j = n//2
  print "j=",j
  delta,f1,g1,T1,P1 = jumpdivsteps(j,j,delta,f,g)
  print "P1:\n" , P1 
  f,g = P1*vector( (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  f,g = kx(f).truncate(t-j),kx(g).truncate(t-j)
  print "truncated: f=",f , "  g=", g
  delta,f2,g2,T2,P2 = jumpdivsteps(n-j,n-j,delta,f,g)
  print "P2:\n" , P2
  f,g = P2*vector( (f,g) )
  print "P2x(f,g): f=" , f , "  g=", g
  f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  print "truncated: f=",f , "  g=", g
  return delta,f,g,T2+T1,P2*P1

# demo code:
f = a2.reverse( 2 )
g = b2.reverse( 2 )
delta, f , g , T2 , T1 = jumpdivsteps( 4 , 5 , 0 , f , g )
print "\ndelta:" , delta, "[f,g] = " , [f,g]
print T2
print T1


jumpdivsteps( 4 5 0 276*xq^2 + 3300*xq + 483 1223*xq^2 + 139*xq + 3443 )
truncated: f= 276*xq^2 + 3300*xq + 483   g= 1223*xq^2 + 139*xq + 3443
j= 2
jumpdivsteps( 2 2 0 276*xq^2 + 3300*xq + 483 1223*xq^2 + 139*xq + 3443 )
truncated: f= 3300*xq + 483   g= 139*xq + 3443
j= 1
jumpdivsteps( 1 1 0 3300*xq + 483 139*xq + 3443 )
truncated: f= 483   g= 3443
P1:
[      1       0]
[1148/xq  483/xq]
P1x(f,g): f= 3300*xq + 483   g= 3688
truncated: f= 483   g= 3688
jumpdivsteps( 1 1 1 483 3688 )
truncated: f= 483   g= 3688
P2:
[      0       1]
[3688/xq 4108/xq]
P2x(f,g): f= 3688   g= 0
truncated: f= 3688   g= 0
P1:
[              1148/xq                483/xq]
[(3688*xq + 1027)/xq^2              852/xq^2]
P1x(f,g): f= 3130*xq + 3688   g= 3277*xq + 2899
truncated: f= 3130*xq + 3688   g= 3277*xq + 2899
jumpdivsteps( 2 2 0 3130*xq + 3688 3277*xq + 2899 )
truncated: f= 3130*xq + 3688   g= 3277*xq + 2899
j= 1
jumpdivsteps( 1 1 0 3130*xq + 3688 3277*xq + 2899 )
truncated: f= 3688   g= 2899
P1:
[      1  

In [82]:
def jumpgcd(R0,R1):
  d = R0.degree()
  assert d>0 and d>R1.degree()
  f,g = R0.reverse(d), R1.reverse(d-1)
  delta,f,g,T,P = jumpdivsteps(2*d-1,3*d-1,1,f,g)
  return f.reverse(delta//2)/f[0]

# demo code:
f = a2.reverse( 2 )
g = b1.reverse( 1 )
jumpgcd(f,g)


jumpdivsteps( 3 5 1 483*xq^2 + 3300*xq + 276 602*xq + 2491 )
truncated: f= 483*xq^2 + 3300*xq + 276   g= 602*xq + 2491
j= 1
jumpdivsteps( 1 1 1 483*xq^2 + 3300*xq + 276 602*xq + 2491 )
truncated: f= 276   g= 2491
P1:
[      0       1]
[2491/xq 4315/xq]
P1x(f,g): f= 602*xq + 2491   g= 311*xq + 1534
truncated: f= 602*xq + 2491   g= 311*xq + 1534
jumpdivsteps( 2 2 0 602*xq + 2491 311*xq + 1534 )
truncated: f= 602*xq + 2491   g= 311*xq + 1534
j= 1
jumpdivsteps( 1 1 0 602*xq + 2491 311*xq + 1534 )
truncated: f= 2491   g= 1534
P1:
[      1       0]
[3057/xq 2491/xq]
P1x(f,g): f= 602*xq + 2491   g= 2736
truncated: f= 2491   g= 2736
jumpdivsteps( 1 1 1 2491 2736 )
truncated: f= 2491   g= 2736
P2:
[      0       1]
[2736/xq 2100/xq]
P2x(f,g): f= 2736   g= 0
truncated: f= 2736   g= 0
P2:
[              3057/xq               2491/xq]
[(2736*xq + 1482)/xq^2             1951/xq^2]
P2x(f,g): f= 2736   g= 3494
truncated: f= 2736   g= 3494


1

In [83]:

# The list version of previous JumpGCD

def jumpdivstep_mat(n,delta,f,g):
  print "jumpdivstep_mat(",n,delta,f,g,")"
  assert n>0
  ### truncate 
  f,g = f[:n], g[:n]
  print "truncated(",n,"): f=",f , "  g=", g
  if n==1:
    rd,f,g,T = divstep_list( delta , f , g )
    return rd,T
  # other cases of n
  j = n//2
  print "j=",j
  delta,P1 = jumpdivstep_mat(j,delta,f,g)
  print "P1:\n" , P1 
  f,g = mat_mul_vec( P1 , (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  delta,P2 = jumpdivstep_mat(n-j,delta,f,g)
  print "P2:\n" , P2
  #f,g = mat_mul_vec( P2 , (f,g) )
  #print "P2x(f,g): f=" , f , "  g=", g
  #f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  #print "truncated: f=",f , "  g=", g
  rP = mat_mul( P2 , P1 )
  print "ret: ", delta , rP
  return delta, rP

def jumpdivsteps_list(n,delta,f,g):
  print "jumpdivsteps_list(",n,delta,f,g,")"
  assert n>0
  if n==1:
    return divstep_list( delta , f , g )
  # other cases of n
  j = n//2
  print "j=",j
  delta,P1 = jumpdivstep_mat(j,delta,f,g)
  print "P1:\n" , P1 
  f,g = mat_mul_vec( P1 , (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  delta,P2 = jumpdivstep_mat(n-j,delta,f,g)
  print "P2:\n" , P2
  f,g = mat_mul_vec( P2 , (f,g) )
  print "P2x(f,g): f=" , f , "  g=", g
  #f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  print "truncated ???: f=",f , "  g=", g
  rP = mat_mul( P2 , P1 )
  return delta, f , g , rP


# demo code:
f = poly_rev( a2 , 2 )
g = poly_rev( b2 , 2 )
jumpdivsteps_list( 5 , 0 , f , g )


jumpdivsteps_list( 5 0 [483, 3300, 276] [3443, 139, 1223] )
j= 2
jumpdivstep_mat( 2 0 [483, 3300, 276] [3443, 139, 1223] )
truncated( 2 ): f= [483, 3300]   g= [3443, 139]
j= 1
jumpdivstep_mat( 1 0 [483, 3300] [3443, 139] )
truncated( 1 ): f= [483]   g= [3443]
P1:
[[[1], [0]], [[1148], [483]]]
P1x(f,g): f= [483, 3300]   g= [3688, 0]
jumpdivstep_mat( 1 1 [483, 3300] [3688, 0] )
truncated( 1 ): f= [483]   g= [3688]
P2:
[[[0], [1]], [[3688], [4108]]]
ret:  0 [[[1148, 0], [483, 0]], [[1027, 3688], [852, 0]]]
P1:
[[[1148, 0], [483, 0]], [[1027, 3688], [852, 0]]]
P1x(f,g): f= [3688, 3130, 0]   g= [2899, 3277, 0]
jumpdivstep_mat( 3 0 [3688, 3130, 0] [2899, 3277, 0] )
truncated( 3 ): f= [3688, 3130, 0]   g= [2899, 3277, 0]
j= 1
jumpdivstep_mat( 1 0 [3688, 3130, 0] [2899, 3277, 0] )
truncated( 1 ): f= [3688]   g= [2899]
P1:
[[[1], [0]], [[1692], [3688]]]
P1x(f,g): f= [3688, 3130, 0]   g= [10, 0, 0]
jumpdivstep_mat( 2 1 [3688, 3130, 0] [10, 0, 0] )
truncated( 2 ): f= [3688, 3130]   g= [10, 0]
j= 

(1,
 [10, 0, 0],
 [0, 0, 0],
 [[[0, 1, 3225, 0, 0], [0, 1932, 38, 0, 0]],
  [[4439, 1874, 4433, 0, 0], [160, 4458, 2059, 0, 0]]])

In [85]:

def jumpgcd_list(R0,R1):
  d = R0.degree()
  assert d>0 and d>R1.degree()
  f = poly_rev(R0,d)
  g = poly_rev(R1,R1.degree())
  g = list_pad_zero(g,len(f))
  delta,f,g,T = jumpdivsteps_list(2*d-1,1,f,g)
  return delta,f,g,T

jumpgcd_list( a2 , b1 )


jumpdivsteps_list( 3 1 [483, 3300, 276] [602, 2491, 0] )
j= 1
jumpdivstep_mat( 1 1 [483, 3300, 276] [602, 2491, 0] )
truncated( 1 ): f= [483]   g= [602]
P1:
[[[0], [1]], [[602], [4108]]]
P1x(f,g): f= [602, 2491, 0]   g= [2977, 876, 0]
jumpdivstep_mat( 2 0 [602, 2491, 0] [2977, 876, 0] )
truncated( 2 ): f= [602, 2491]   g= [2977, 876]
j= 1
jumpdivstep_mat( 1 0 [602, 2491] [2977, 876] )
truncated( 1 ): f= [602]   g= [2977]
P1:
[[[1], [0]], [[1614], [602]]]
P1x(f,g): f= [602, 2491]   g= [2736, 0]
jumpdivstep_mat( 1 1 [602, 2491] [2736, 0] )
truncated( 1 ): f= [602]   g= [2736]
P2:
[[[0], [1]], [[2736], [3989]]]
ret:  0 [[[1614, 0], [602, 0]], [[1664, 2736], [285, 0]]]
P2:
[[[1614, 0], [602, 0]], [[1664, 2736], [285, 0]]]
P2x(f,g): f= [2736, 0, 0]   g= [2332, 0, 0]
truncated ???: f= [2736, 0, 0]   g= [2332, 0, 0]


(0,
 [2736, 0, 0],
 [2332, 0, 0],
 [[[4306, 0, 0], [3058, 1614, 0]], [[1703, 0, 0], [75, 1664, 2736]]])